In [1]:
import tensorflow as tf
import numpy as np
import cv2
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [3]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

In [4]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [5]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [6]:
cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture("cmj.mp4")
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    
    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.3)
    draw_keypoints(frame, keypoints_with_scores, 0.3)
    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

### SAVING OUTPUT FILE

In [2]:
import pickle

In [29]:
def save_to_file(data_dic, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(data_dic, f)

        # for key in data_dic.keys():
        #     pickle.dump(key, f)
        #     pickle.dump(data_dic[key], f)

In [39]:
fs = "hhhh"
v_info = "skjdfld"
kp_vec = [1, 3, 3]
kp_w_scores_vec = [1, 2, 3, 4, 5]
video_time = 10

In [40]:
data = {
    'fs_info': {'fs':fs},
    'v_info': v_info,
    'kp': kp_vec,
    'kp_w_scores_vec': kp_w_scores_vec,
    'video_time': video_time
}

In [41]:
save_to_file(data, 'aquivo')   


In [42]:
infile = open('aquivo','rb')
new_dict = pickle.load(infile)
infile.close()


In [43]:
print(new_dict)

{'fs_info': {'fs': 'hhhh'}, 'v_info': 'skjdfld', 'kp': [1, 3, 3], 'kp_w_scores_vec': [1, 2, 3, 4, 5], 'video_time': 10}
